In [1]:
import ts from "typescript";
import { requireCytoscape, requireCarbon, linePlot, draw, drawMemTrace, memLayout } from "./lib/draw";

# Review

## Algebraic data-types

```
type tree<T> = 
    { tag: "LEAF" }
  | { tag: "NODE", contents: T, left: tree<T>, right: tree<T> };
```

## Recursion

```
function f(x) {
   ... f(smaller x);
}
```

```
max([1, 2, 3, 4, 5, 6])
  max(1, max([2, 3, 4, 5, 6]))

  max(max([1, 2, 3]), max([4, 5, 6]))
```

```
sort([1, 2, 3, 4, 5, 6])
merge(sort([1, 2, 3]), sort([4, 5, 6]))
merge([3,2,1], [6,5,4])

sort([1, 2, 3])
   merge(sort([1, 2]), sort([3]))
```



## First-class functions

```
(x: number) => (y: (z: number) => number) => y(x)

function map<T, U>(f: (x: T) => U): U[] {
   ...
}

function filter<T>(f: (x: T) => boolean): T[] {
   ...
}

function reduce<T, U>(f: (acc: U, x:T) => U): U {
   ...
}

```

## Pure vs. impure functions

```
pure(1) = 2
pure(1) = 2
pure(1) = 2
pure(1) = 2
pure(1) = 2
pure(1) = 2

question: what does pure(1) give?

impure(1) = 2
impure(1) = 3
impure(1) = 3
impure(1) = 3
impure(1) = 2
impure(1) = 1
impure(1) = 5

question: what does impure(1) give?
```

## Closures

```
onClick(callback);

let counter = 0;
function callback(x) {
  .. use x and counter
}
```

## LeetCode Problem 30: Substring with Concatenation of All Words

27.1% Hard

You are given a string s and an array of strings words of the same length.
Return all starting indices of substring(s) in s that is a concatenation of each word
in words exactly once, in any order, and without any intervening characters.

You can return the answer in any order.

### Example 1:

```
Input: s = "barfoothefoobarman", words = ["foo","bar"]
Output: [0,9]
Explanation: Substrings starting at index 0 and 9 are "barfoo" and "foobar" respectively.
The output order does not matter, returning [9,0] is fine too.
```

### Example 2:

```
Input: s = "wordgoodgoodgoodbestword", words = ["word","good","best","word"]
Output: []
```

### Example 3:

```
Input: s = "barfoofoobarthefoobarman", words = ["bar","foo","the"]
Output: [6,9,12]
```

### Example 4:

```
Input: s = "anttan", words = ["ant", "tan"]
Output: [0]
```

In [2]:
const s = "barfoothefoobarman";
const words = ["foo","bar"];
const s2 = "wordgoodgoodgoodbestword";
const words2 = ["word","good","best","word"];
const s3 = "barfoofoobarthefoobarman";
const words3 = ["bar","foo","the"];
const s4 = "lingmindraboofooowingdingbarrwingmonkeypoundcake";
const words4 = ["fooo","barr","wing","ding","wing"];

```
findSubstring("barfoothefoobarman", ["foo","bar"]);
  findSubstring("foothefoobarman", ["foo"]);
    findSubstring("thefoobarman", []);
      - in my base case, []
```

```
findSubstring(["bar", "foo", "the", "foo", "bar", "man"], ["foo","bar"]);
  findSubstring(["foo", "the", "foo", "bar", "man"], ["foo"]);
    findSubstring(["the", "foo", "bar", "man"], []);
      - in my base case, []
```
      

In [3]:
type lcstr = 
    { tag: "LEAF" } // ""
  | { tag: "WORD", word: string, idx: number, rest: lcstr };

function Leaf(): lcstr {
    return { tag: "LEAF" };
}

function Word(word: string, idx: number, rest: lcstr): lcstr {
    return { tag: "WORD", word: word, idx: idx, rest: rest };
}

function lcstrToString(s: lcstr) {
    switch (s.tag) {
        case "LEAF": {
            return "";
        }
        case "WORD": {
            return s.word + s.idx + lcstrToString(s.rest);
        }
    }
}

In [4]:
function stringToLcstr(s: string, len: number): lcstr {
    function go(s: string, idx: number) {
        if (s.length === 0) {
            return Leaf();
        } else {
            return Word(s.substring(0, len), idx, go(s.substring(len), idx + len));
        }
    }
    return go(s, 0);
}

// const s = "barfoothefoobarman";
// const words = ["foo","bar"];

lcstrToString(stringToLcstr(s, 3))
// stringToLcstr(s, 3)

bar0foo3the6foo9bar12man15


In [5]:
// findSubstring(["bar", "foo", "the", "foo", "bar", "man"], ["foo","bar"]);
//   findSubstring(["foo", "the", "foo", "bar", "man"], ["foo"]);
//     findSubstring(["the", "foo", "bar", "man"], []);
//       - in my base case, []

function search(start: number, s: lcstr, words: string[]) {
    switch (s.tag) {
        case "LEAF": {
            return words.length === 0 ? start : undefined;
        }
        case "WORD": {
            if (words.length === 0){
                return start;
            } else {
                for (let i = 0; i < words.length; i++) {
                    if (words[i] === s.word) {
                        start = start === undefined ? s.idx : start;
                        const smallerWords = words.slice(0, i).concat(words.slice(i + 1));
                        return search(start, s.rest, smallerWords);
                    }
                }
                return undefined;
            }
        }
    }
}

In [6]:
const arr = [1, 2, 3, 4];
const i = 2;
const left = arr.slice(0, i) // [1, 2]
const right = arr.slice(i + 1) // [4]
console.log(left)
console.log(right)
left.concat(right)

[ 1, 2 ]
[ 4 ]
[ 1, 2, 4 ]


In [7]:
const s = "barfoothefoobarman";
const words = ["foo","bar"];
search(0, stringToLcstr(s, 3), words)

0


In [8]:
function go(acc: number[], lc: lcstr) {
    switch (lc.tag) {
        case "LEAF": {
            return acc;
        }
        case "WORD": {
            const res = search(undefined, lc, words);
            if (res !== undefined) {
                acc.push(res);
                return go(acc, lc.rest);
            } else {
                return go(acc, lc.rest);
            }
        }
    }
}


In [9]:
function findSubstring(s: string, words: string[]): number[] {
    if (words.length === 0) {
        return [];
    }

    // 1. Create a model
    type lcstr = 
        { tag: "LEAF" }
      | { tag: "WORD", word: string, idx: number, rest: lcstr };

    function Leaf(): lcstr {
        return { tag: "LEAF" };
    }

    function Word(word: string, idx: number, rest: lcstr): lcstr {
        return { tag: "WORD", word: word, idx: idx, rest: rest };
    }
    
    // 2. Solved a reduced problem
    function search(start: number, s: lcstr, words: string[]) {
        switch (s.tag) {
            case "LEAF": {
                return words.length === 0 ? start : undefined;
            }
            case "WORD": {
                if (words.length === 0){
                    return start;
                } else {
                    for (let i = 0; i < words.length; i++) {
                        if (words[i] === s.word) {
                            start = start === undefined ? s.idx : start;
                            return search(start, s.rest, words.slice(0, i).concat(words.slice(i + 1)));
                        }
                    }
                    return undefined;
                }
            }
        }
    }
    
    // 3. Used the reduced problem to solve the actual problem
    function go(acc: number[], lc: lcstr) {
        switch (lc.tag) {
            case "LEAF": {
                return acc;
            }
            case "WORD": {
                const res = search(undefined, lc, words);
                if (res !== undefined) {
                    acc.push(res);
                    return go(acc, lc.rest);
                } else {
                    return go(acc, lc.rest);
                }
            }
        }
    }
    
//     const s3 = "bar foo foo bar the foo bar man";
//     const words3 = ["bar","foo","the"];
    let acc: number[][] = [];
    for (let i = 0; i < words[0].length; i++) {
        // i = 0 "go(barfoofoobarthefoobarman, ["bar","foo","the"])";
        // i = 1 "go(arfoofoobarthefoobarman, ["bar","foo","the"])";
        // i = 2 "go(rfoofoobarthefoobarman)";
        acc.push(go([], stringToLcstr(s.substring(i), words[0].length)).map((x: number) => x + i))
    }
    
    return acc.reduce((acc, x) => acc.concat(x));
}



In [10]:
function timeFunction(name, f) {
    console.log(`--------------------------`);
    console.log(`${name} started..`);
    const t0 = process.hrtime()
    f();
    const t1 = process.hrtime(t0);
    console.log(`${f.name} completed..`);
    console.info('Execution time (hr): %ds %dms', t1[0], t1[1] / 1000000);
    return t1[0] + t1[1] / 1000000 / 1000;
}


In [11]:
const counts = [];
const times = [];
for (let i = 1; i < 10; i++) {
    let s = "";
    const words = [];
    for (let j = 0; j < i*10; j++) {
        s += "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa";
        for (let k = 0; k < s.length; k++) {
            words.push("a");
        }
        
    }
    counts.push(s.length);
    times.push(timeFunction("findSubstring", () => findSubstring(s3, words)));
}

linePlot(counts, [times])

--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 2.569417ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 3.736458ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 2.53375ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 4.118458ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 8.210708ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 9.178417ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 17.545875ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 17.332708ms
--------------------------
findSubstring started..
 completed..
Execution time (hr): 0s 21.104792ms
